In [ ]:
!pip install momepy
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import momepy as mm
import osmnx as ox
import numpy as np
from shapely.geometry import box
import utilsALL

ModuleNotFoundError: No module named 'utilsALL'

In [ ]:
gdf = gpd.read_file('ALLNAIR.shp')
BNEWAOI= gdf.to_crs(32737)
BNEWAOI.crs

<Derived Projected CRS: EPSG:32737>
Name: WGS 84 / UTM zone 37S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, southern hemisphere between 80°S and equator, onshore and offshore. Kenya. Mozambique. Tanzania.
- bounds: (36.0, -80.0, 42.0, 0.0)
Coordinate Operation:
- name: UTM zone 37S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
BNEWAOI.drop(columns=['TARGET_FID', 'full_plus_', 'confidence', 'Shape_Area' , 'Shape_Leng' , 'Join_Count'], inplace=True)

## Preprocessing Buildings

In [ ]:
BNEWAOI.geometry = BNEWAOI.buffer(0)
BNEWAOI= BNEWAOI[~BNEWAOI.geometry.isna()]

In [ ]:
BNEWAOI.geom_type.value_counts()

Polygon         825445
MultiPolygon        12
dtype: int64

In [ ]:
BNEWAOI = BNEWAOI.reset_index(drop=True).explode(index_parts=True).reset_index(drop=True)
BNEWAOI.geom_type.value_counts()

In [ ]:
BNEWAOI.columns

Index(['latitude', 'longitude', 'area_in_me', 'HeightPROP', 'geometry'], dtype='object')

In [ ]:
duplicate_index = BNEWAOI.index.duplicated().any()
if duplicate_index:
    print("Some rows have the same index.")
else:
    print("All rows have unique indices.")

All rows have unique indices.


In [ ]:
BNEWAOI = BNEWAOI[BNEWAOI.area > 5]

In [ ]:
missing_height_count = BNEWAOI['HeightPROP'].isin([0, np.nan]).sum()
print(f"Number of missing height values: {missing_height_count}")

Number of missing height values: 22460


In [ ]:
rows_with_missing_height = BNEWAOI[BNEWAOI['HeightPROP'].isin([0, np.nan])]
print("\nRows with missing height values:")
print(rows_with_missing_height)


Rows with missing height values:
        latitude  longitude  area_in_me  HeightPROP  \
19     -1.264190  37.090509     73.5421         0.0   
56     -1.275440  36.712186    458.6360         0.0   
156    -1.239985  36.749919     91.4232         0.0   
172    -1.316951  36.678537     13.6525         0.0   
179    -1.290601  37.007219     11.2587         0.0   
...          ...        ...         ...         ...   
825297 -1.252996  36.974519     27.4398         0.0   
825400 -1.273251  36.715150    143.0698         0.0   
825418 -1.372106  36.749473    126.9240         0.0   
825427 -1.293595  36.908769     35.9132         0.0   
825435 -1.236162  36.853721     23.1514         0.0   

                                                 geometry  
19      POLYGON ((287539.486 9860188.195, 287535.166 9...  
56      POLYGON ((245421.625 9858896.218, 245405.804 9...  
156     POLYGON ((249616.745 9862827.832, 249612.063 9...  
172     POLYGON ((241678.451 9854317.630, 241677.216 9...  
179  

In [ ]:
mean_height = BNEWAOI['HeightPROP'].mean()
BNEWAOI.loc[BNEWAOI['HeightPROP'] == 0, 'HeightPROP'] = mean_height

In [ ]:
empty_height = BNEWAOI['HeightPROP'].isin([0, np.nan]).sum()
if empty_height:
    print("Null values still exist in the 'height' column.")
else:
    print("No null values in the 'height' column.")

No null values in the 'height' column.


In [ ]:
BNEWAOI["uID"] = range(len(BNEWAOI))
BNEWAOI['Height'] = BNEWAOI['HeightPROP'].round(2)
BNEWAOI.drop(columns=['HeightPROP'], inplace=True)
BNEWAOI.head()

,latitude,longitude,area_in_me,HeightPROP,geometry,uID
0,-1.202541,36.881486,34.9328,9.1,"POLYGON ((264263.851 9866989.675, 264258.853 9...",0
1,-1.332171,36.858919,14.0811,7.6,"POLYGON ((261761.782 9852652.634, 261760.765 9...",1
2,-1.194113,36.910146,41.5347,6.6,"POLYGON ((267454.398 9867926.728, 267450.879 9...",2
3,-1.248877,36.885837,94.2778,8.9,"POLYGON ((264754.659 9861870.127, 264754.005 9...",3
4,-1.290072,36.737170,7.3208,3.7,"POLYGON ((248202.552 9857294.537, 248200.127 9...",4


In [ ]:
BNEWAOI.to_parquet('CleanBlgALL.pq')

## Generate tessellation

In [ ]:
Building = BNEWAOI

In [ ]:
checkVBuilding = mm.CheckTessellationInput( Building, shrink = 0.6)
checkV3 = mm.CheckTessellationInput(BNEWAOI_Tessinput, shrink = 0.6)

Collapsed features  : 181
Split features      : 63
Overlapping features: 215


In [ ]:
BUILD = Building.drop(checkVBuilding.overlap.index.union(checkVBuilding.split.index))

In [ ]:
from shapely.geometry import MultiPolygon
multi_polygon_uids = []
for index, row in BUILD.iterrows():
    if isinstance(row.geometry, MultiPolygon):
        multi_polygon_uids.append(row['uID'])
print("UIDs of MultiPolygons:", multi_polygon_uids)

UIDs of MultiPolygons: []


In [ ]:
BUILD["uID"] = range(len(BUILD))
BUILD.head()

,latitude,longitude,area_in_me,geometry,uID,Height
0,-1.202541,36.881486,34.9328,"POLYGON ((264263.851 9866989.675, 264258.853 9...",0,9.1
1,-1.332171,36.858919,14.0811,"POLYGON ((261761.782 9852652.634, 261760.765 9...",1,7.6
2,-1.194113,36.910146,41.5347,"POLYGON ((267454.398 9867926.728, 267450.879 9...",2,6.6
3,-1.248877,36.885837,94.2778,"POLYGON ((264754.659 9861870.127, 264754.005 9...",3,8.9
4,-1.290072,36.737170,7.3208,"POLYGON ((248202.552 9857294.537, 248200.127 9...",4,3.7


In [ ]:
BUILD = gpd.read_parquet('CleanBlgALL.pq')

In [ ]:
limit = mm.buffered_limit(BUILD, 100)
%time TessALL = mm.Tessellation(BUILD, "uID", limit, segment = 0.7 , shrink = 0.6).tessellation

Inward offset...
Generating input point array...
Generating Voronoi diagram...
Generating GeoDataFrame...
Dissolving Voronoi polygons...


/usr/local/lib/python3.8/dist-packages/momepy/elements.py:384: UserWarning: Tessellation does not fully match buildings. 182 element(s) collapsed during generation - unique_id: {328705, 510982, 256024, 509982, 598051, 654371, 113706, 411704, 578626, 212035, 385091, 164933, 772171, 258124, 555087, 692319, 222308, 199790, 121969, 453750, 639105, 174223, 378004, 484508, 423071, 479393, 92322, 239781, 197799, 210093, 279734, 491712, 799939, 505028, 603332, 669895, 55496, 321737, 548044, 823500, 6354, 719059, 278740, 184533, 450776, 126171, 174303, 802028, 366846, 269568, 370945, 445696, 555265, 379148, 323853, 204046, 793868, 25878, 727328, 336174, 800047, 437582, 478544, 89425, 588115, 415060, 823635, 467293, 269663, 388449, 736609, 365941, 278905, 29050, 815483, 508284, 305534, 345475, 755085, 681361, 741786, 386459, 70046, 36259, 356787, 144827, 284092, 418256, 213463, 202202, 201181, 148970, 822768, 218638, 296466, 562716, 170525, 652829, 705057, 620067, 155175, 413227, 26163, 377398, 

CPU times: user 7h 25min 29s, sys: 8min 51s, total: 7h 34min 21s
Wall time: 7h 34min 16s


In [ ]:
BUILDS.to_parquet('CleanBlgNAOI.pq')
BUILDS.to_file('CleanBlgNAOI.shp')
TessALL.to_file('TessALL.shp')
TessALL.to_parquet('TessALL.pq')